In [10]:
using JuMP
using CPLEX;

In [9]:
function extraire_array(str)
    matches = match(r"\[([\d,\s]+)\]", str)
# Если нашли совпадение, извлекаем числа и преобразуем их в массив
    if matches !== nothing
        p_values = parse.(Int64, split(matches.captures[1], ","))
        return p_values;
    end
end

function readfile(namefile::String)

    n=0;
    s=0;
    t=0;
    S=0;
    d1=0;
    d2=0;
    p=[];
    ph=[];
    flag = false;
    AdjMat=[];

    d=[];
    D=[];

    if isfile(namefile)
        file=open(namefile,"r")
        data=readlines(file);
        for line in data
            str=split(line," ")
            if (flag) ## on lit matrice d'adjeccance
                str[end]=chop(str[4])
                numbers = parse.(Int64,str[1:3]);
                d[numbers[1],numbers[2]]=numbers[3];
                AdjMat[numbers[1],numbers[2]]=1;
                D[numbers[1],numbers[2]]=parse(Float64,str[4]);
            end
            if (str[1]=="n")
                n=parse(Int64,str[3]);
                d=zeros(Int64,n,n);
                D=zeros(Float64,n,n);
                AdjMat=zeros(Int64,n,n);
            end

            if (str[1]=="s")
                s=parse(Int64,str[3]);
            end
            
            if (str[1]=="t")
                t=parse(Int64,str[3]);
            end
            if (str[1]=="S")
                S=parse(Int64,str[3])
            end

            if (str[1]=="d1" )
                d1=parse(Int64,str[3]);
            end
            if (str[1]=="d2")
                d2=parse(Int64,str[3]);
            end
            
            if (str[1]=="p")
                p=extraire_array(join(str[3:end], " "))
            end

            if (str[1]=="ph")
                ph=extraire_array(join(str[3:end], " "))
            end

            if (str[1]=="Mat")
                flag= true;
            end
        end
    end
    return(n,s,t,S,d1,d2,p,ph,d,D,AdjMat)
end

readfile (generic function with 1 method)

In [8]:
function dataFiles(dirName::String) #ecrit tous les noms des fichoer qu ont .gr 
    names=[]
    for file in readdir(dirName)
        if occursin(".gr",file)
            push!(names,file)
        end
    end
    return names
end

dataFiles (generic function with 1 method)

In [10]:
#statique function 
function StatModel(n,s,t,S,d1,d2,p,ph,d,D,AdjMat) #return obj value

    model=Model(CPLEX.Optimizer)
    set_optimizer_attribute(model, "CPX_PARAM_SCRIND", 0)
    start = time()
    
    #Variables
    @variable(model,x[i in 1:n,j in 1:n;AdjMat[i,j]==1],Bin)
    @variable(model,y[1:n],Bin)

    #Constraintes
    @constraint(model,sum(p[i]*y[i] for i in 1:n)<=S)
    @constraint(model,[i in 1:n;i!=t], y[i]==sum(x[i,j] for j in 1:n if AdjMat[i,j]==1))
    @constraint(model,y[t]==sum(x[i,t] for i in 1:n if AdjMat[i,t]==1))
    @constraint(model,[i in 1:n;i!=s&&i!=t], 
        sum(x[j,i] for j in 1:n if AdjMat[j,i]==1) - sum(x[i,j] for j in 1:n if AdjMat[i,j]==1)==0)
    @constraint(model,sum(x[s,j] for j in 1:n if AdjMat[s,j]==1) - sum(x[j,s] for j in 1:n if AdjMat[j,s]==1)==1)
    @constraint(model,sum(x[t,j] for j in 1:n if AdjMat[t,j]==1) - sum(x[j,t] for j in 1:n if AdjMat[j,t]==1)==-1)

    #objectif
    @objective(model,Min,sum(d[i,j]*x[i,j] for i in 1:n,j in 1:n if AdjMat[i,j]==1))

    optimize!(model)
    computation_time = time() - start

    # Récupération du status de la résolution
    feasibleSolutionFound = primal_status(model) == MOI.FEASIBLE_POINT
    isOptimal = termination_status(model) == MOI.OPTIMAL
    if feasibleSolutionFound
        println("Probleme resolu:")
        #Valeur objectif
        if isOptimal 
            println("Valeur de l'objectif : ",JuMP.objective_value(model))
            return JuMP.objective_value(model), computation_time;
        end
    end
    

end

StatModel (generic function with 1 method)

In [11]:
# Resoltion du probleme statique

outFile="statique_result4.csv"


filesNames=["2300_USA-road-d.COL.gr",
"1800_USA-road-d.COL.gr"
]

for name in filesNames
    println(name)
    fout=open(outFile,"a")
    n,s,t,S,d1,d2,p,ph,d,D,AdjMat=readfile("Data_project\\data\\processed\\"*name)
    objValue,time=StatModel(n,s,t,S,d1,d2,p,ph,d,D,AdjMat)
    println(fout,name*",",objValue,",",time)
    close(fout)
end


2300_USA-road-d.COL.gr
Probleme resolu:

Valeur de l'objectif : 40803.0


1800_USA-road-d.COL.gr


Probleme resolu:
Valeur de l'objectif : 34075.0


In [11]:
function SP2(y, n,s,t,S,d1,d2,p,ph,d,D,AdjMat)# return (objective_value, delta2, flag)

    m2=Model(CPLEX.Optimizer)
    set_optimizer_attribute(m2, "CPX_PARAM_TILIM", 10)
    set_optimizer_attribute(m2, "CPX_PARAM_SCRIND", 0)
    #variable delta_ij
    @variable(m2,delta2[i in 1:n]>=0)

    #Contraintes
    @constraint(m2, [i in 1:n], delta2[i]<=2);
    @constraint(m2,sum(delta2[i] for i in 1:n)<=d2)

    #Objectif
    @objective(m2,Max,sum(y[i]*(p[i]+delta2[i]*ph[i]) for i in 1:n));

    optimize!(m2)

    # Récupération du status de la résolution
    feasibleSolutionFound = primal_status(m2) == MOI.FEASIBLE_POINT
    isOptimal = termination_status(m2) == MOI.OPTIMAL
    if feasibleSolutionFound
        #println("Sous-Probleme1 resolu:")
        #Valeur objectif
        #println("Valeur de l'objectif : ",JuMP.objective_value(m2))
        return (JuMP.objective_value(m2),JuMP.value.(delta2),1)
    else
        return (-1,[],0)
    end
    
end

SP2 (generic function with 1 method)

In [12]:
#return (obj_value, delta1, flag) - flag=1 si on a trouver la solution ot, 0 sinon
function SP1(x, n,s,t,S,d1,d2,p,ph,d,D,AdjMat)

    m1=Model(CPLEX.Optimizer)
    set_optimizer_attribute(m1, "CPX_PARAM_TILIM", 10)
    set_optimizer_attribute(m1, "CPX_PARAM_SCRIND", 0)
    #variable delta_ij
    @variable(m1,delta1[i in 1:n, j in 1:n;AdjMat[i,j]==1]>=0)

    #Contraintes
    @constraint(m1, [i in 1:n, j in 1:n; AdjMat[i,j]==1], delta1[i,j]<=D[i,j] )
    @constraint(m1,sum(delta1[i,j] for i in 1:n, j in 1:n if AdjMat[i,j]==1)<=d1)

    #Objectif
    @objective(m1,Max,sum(x[i,j]*d[i,j]*(1+delta1[i,j]) for i in 1:n, j in 1:n if AdjMat[i,j]==1))

    optimize!(m1)

    # Récupération du status de la résolution
    feasibleSolutionFound = primal_status(m1) == MOI.FEASIBLE_POINT
    isOptimal = termination_status(m1) == MOI.OPTIMAL
    if feasibleSolutionFound 
        #println("Sous-Probleme1 resolu:")
        #Valeur objectif
        #println("Valeur de l'objectif : ",JuMP.objective_value(m1))
        return (JuMP.objective_value(m1),JuMP.value.(delta1[:,:]),1)
    else
        return (-1,-1,0)
    end

end

SP1 (generic function with 1 method)

1-element Vector{String}:
 "1600_USA-road-d.BAY.gr"

In [14]:
# On précise que le modèle doit utiliser notre fonction de callback
resoudre=0
filesNames=dataFiles("Data_project\\data\\processed\\")
for name in filesNames
     n,s,t,S,d1,d2,p,ph,d,D,AdjMat=readfile("Data_project\\data\\processed\\"*name)

     model_BC=Model(CPLEX.Optimizer)
     MOI.set(model_BC, MOI.NumberOfThreads(), 1)
     set_optimizer_attribute(model_BC, "CPX_PARAM_TILIM", 100)
     set_optimizer_attribute(model_BC, "CPX_PARAM_SCRIND", 0)


     function mon_callback(cb_data::CPLEX.CallbackContext, context_id::Clong)
          # On teste d’abord si c’est l’obtention d’une solution entière
          # qui a entraîné l’appel du callback
          # (cette fonction isIntegerPoint est définie ci-dessous mais son
          # contenu n’est pas très important)
      
          if isIntegerPoint(cb_data, context_id)
              # Cette ligne doit être appelée avant de pouvoir récupérer la
              # solution entière ayant entraîné l’appel du callback
              CPLEX.load_callback_variable_primal(cb_data, context_id)
              # On récupère la valeur de x
              x_val = callback_value.(cb_data, model_BC[:x][:,:])
              z_val = callback_value(cb_data, z)
              y_val = callback_value.(cb_data, model_BC[:y])
      
              z_1,delta1,flag1=SP1(x_val, n,s,t,S,d1,d2,p,ph,d,D,AdjMat);
      
              z_2,delta2,flag2=SP2(y_val, n,s,t,S,d1,d2,p,ph,d,D,AdjMat);
      
              if flag1!=1 || flag2!=1
                  println("Don't know what to do")
              end
      
              # on ajoute epsilon 1e-5
                  if abs(z_1-z_val)> 1e-5
                      cstr = @build_constraint(model_BC[:z] >= sum(d[i,j]*(delta1[i,j]+1)*(model_BC[:x])[i,j] for i in 1:n, j in 1:n if AdjMat[i,j]==1))
                      MOI.submit(model_BC, MOI.LazyConstraint(cb_data), cstr)
          
                      #println("Add constraint type 1")
                  end
                  if (z_2> S+1e-5)
                      cstr=@build_constraint(S>= sum((model_BC[:y])[i]*(delta2[i]*ph[i]+p[i]) for i in 1:n))
                      MOI.submit(model_BC,MOI.LazyConstraint(cb_data),cstr)
      
                     # println("Add constraint type 2")
                  end
              
          end
      end
      
      # Fonction permettant de déterminer si c’est l’obtention d’une
      # solution entière qui a entraîné l’appel d’un callback
      # (il n’est pas nécessaire d’en comprendre le fonctionnement)1
          function isIntegerPoint(cb_data::CPLEX.CallbackContext, context_id::Clong)
          # context_id == CPX_CALLBACKCONTEXT_CANDIDATE si le callback est
          # appelé dans un des deux cas suivants :
          # cas 1 - une solution entière a été obtenue; ou
          # cas 2 - une relaxation non bornée a été obtenue
          if context_id != CPX_CALLBACKCONTEXT_CANDIDATE
              return false
          end
          # Pour déterminer si on est dans le cas 1 ou 2, on essaie de récupérer la
          # solution entière courante
          ispoint_p = Ref{Cint}()
          ret = CPXcallbackcandidateispoint(cb_data, ispoint_p)
          # S’il n’y a pas de solution entière
          if ret != 0 || ispoint_p[] == 0
              return false
          else
              return true
          end
      end
              

     @variable(model_BC,x[1:n ,1:n], Bin)
     @variable(model_BC,y[1:n], Bin)
     @variable(model_BC,z>=0)

     # Contraint de base
     @constraint(model_BC,[j in 1:n; j!=t], y[j]==sum( x[i,j] for i in 1:n if AdjMat[i,j]==1))
     @constraint(model_BC,y[t]==sum(x[j,t] for j in 1:n if AdjMat[j,t]==1))
     @constraint(model_BC,sum(x[s,j] for j in 1:n if AdjMat[s,j]==1) - sum(x[j,s] for j in 1:n if AdjMat[j,s]==1) ==1)
     @constraint(model_BC,sum(x[t,j] for j in 1:n if AdjMat[t,j]==1) - sum(x[j,t] for j in 1:n if AdjMat[j,t]==1) == -1)

     @constraint(model_BC, [i in 1:n;i != s && i != t],sum(x[i,j] for j in 1:n if AdjMat[i,j]== 1)
          - sum(x[j, i] for j in 1:n if AdjMat[j,i]==1) == 0)
     

     #Contrainte de Master Probleme
     @constraint(model_BC, sum(x[i,j]*d[i,j] for i in 1:n, j in 1:n if AdjMat[i,j]==1)<=z)
     @constraint(model_BC, sum(y[i]*p[i] for i in 1:n)<=S)

     # objectif function
     @objective(model_BC,Min,z)

     MOI.set(model_BC, CPLEX.CallbackFunction(), mon_callback)
     optimize!(model_BC)

     feasibleSolutionFound = primal_status(model_BC) == MOI.FEASIBLE_POINT
     isOptimal = termination_status(model_BC) == MOI.OPTIMAL
     if feasibleSolutionFound
          if isOptimal
            resoudre = rsoudre +1
               println(JuMP.objective_value(model_BC),",optimal,")
          #Valeur objectif
          else
               println(JuMP.objective_value(model_BC),",feasible,")
          end
    else
        print(name,",-1,not find,")
     end
     
end

println(resoudre, " 100s")

In [42]:
#Heuristique

#1 On resous dans le pire cas

function Heurist1(n,s,t,S,d1,d2,p,ph,d,D,AdjMat)

    h1=Model(CPLEX.Optimizer)

    delta_values = zeros(n, n)

        # Loop over each pair (i, j) and randomly choose a value for delta_ij
    for i in 1:n
        for j in 1:n
            if AdjMat[i,j]==1
            # Generate a random value for delta_ij in the range [0, D_ij]
                delta_values[i, j] = rand() * D[i, j]
            end
        end
    end

    if sum(delta_values) <= d1
        println("Randomly chosen values satisfy the constraint.")
    else
        println("Randomly chosen values do not satisfy the constraint.")
    end


    delta2_r=[rand()*2 for i in 1:n]
    if sum(delta2_r)<=d2
        println("delta2 bon")
    else
        println("We need retry delta2")
    end


    
    #Solve with these delta1 et delta_2 

end

## Resulting table and Plots


In [1]:
using Pkg
Pkg.add("DataFrames")


┌ Warning: attempting to remove probably stale pidfile
│   path = C:\Users\Boss\.julia\registries\.pid
└ @ FileWatching.Pidfile C:\Users\Boss\AppData\Local\Programs\Julia-1.9.3\share\julia\stdlib\v1.9\FileWatching\src\pidfile.jl:244


    Updating registry at `C:\Users\Boss\.julia\registries\General.toml`


   Resolving package versions...


   Installed JpegTurbo ───────────── v0.1.5


   Installed FFTW ────────────────── v1.8.0
   Installed Permutations ────────── v0.4.20


...

  ✓ DelaunayTriangulation


  ✓ StatsPlots


  ✓ Makie


  ✓ CairoMakie


  254 dependencies successfully precompiled in 772 seconds. 207 already precompiled.
  2 dependencies had warnings during precompilation:
┌ MKL_jll [856f044c-d86e-5d09-b602-aeab76dc8ba7]
│   Downloading artifact: MKL
└  
┌ Interpolations [a98d9a8b-a2ab-59e6-89dd-64a1c18fca59]
│  WARNING: method definition for checkbounds at C:\Users\Boss\.julia\packages\Interpolations\USkTk\src\Interpolations.jl:454 declares type variable N but does not use it.
│  WARNING: method definition for checkbounds at C:\Users\Boss\.julia\packages\Interpolations\USkTk\src\Interpolations.jl:457 declares type variable N but does not use it.
│  WARNING: method definition for GriddedInterpolation at C:\Users\Boss\.julia\packages\Interpolations\USkTk\src\gridded\gridded.jl:37 declares type variable pad but does not use it.
│  WARNING: method definition for GriddedInterpolation at C:\Users\Boss\.julia\packages\Interpolations\USkTk\src\gridded\gridded.jl:60 declares type variable pad but does not use it.
│  WARNING: m

In [9]:
using CSV
using DataFrames

r_stat=DataFrame(CSV.File("statique_result4.csv", header=false))
total=DataFrame()

total.name=r_stat[:,1]
total.stat_value=r_stat[:,2]
total.time_stat=r_stat[:,3]


123-element Vector{Float64}:
    3.76200008392334
    5.763999938964844
    2.7780001163482666
    0.08500003814697266
    0.059999942779541016
    0.04700016975402832
    5.13100004196167
    9.958999872207642
    6.309000015258789
    8.375999927520752
    ⋮
    4.611000061035156
    2.117000102996826
  151.31500005722046
   35.71399998664856
   74.7720000743866
  129.9359998703003
  326.74399995803833
 1397.6740000247955
  677.2589998245239

In [10]:
r_dualisation=DataFrame(CSV.File("out_result_v3.csv",header=false),["name","nonuse","dual_val"])


r_dualisation[1:3,:]

┌ Warning: thread = 1 warning: only found 2 / 3 columns around data row: 20. Filling remaining columns with `missing`
└ @ CSV C:\Users\Boss\.julia\packages\CSV\OnldF\src\file.jl:577
┌ Warning: thread = 1 warning: only found 2 / 3 columns around data row: 26. Filling remaining columns with `missing`
└ @ CSV C:\Users\Boss\.julia\packages\CSV\OnldF\src\file.jl:577
┌ Warning: thread = 1 warning: only found 2 / 3 columns around data row: 27. Filling remaining columns with `missing`
└ @ CSV C:\Users\Boss\.julia\packages\CSV\OnldF\src\file.jl:577
┌ Warning: thread = 1 warning: only found 2 / 3 columns around data row: 29. Filling remaining columns with `missing`
└ @ CSV C:\Users\Boss\.julia\packages\CSV\OnldF\src\file.jl:577
┌ Warning: thread = 1 warning: only found 2 / 3 columns around data row: 35. Filling remaining columns with `missing`
└ @ CSV C:\Users\Boss\.julia\packages\CSV\OnldF\src\file.jl:577
┌ Warning: thread = 1 warning: only found 2 / 3 columns around data row: 38. Filling remai

Row,name,nonuse,dual_val
,String31,String7?,Float64?
1,1000_USA-road-d.BAY.gr,value,48250.0
2,1000_USA-road-d.COL.gr,value,29672.8
3,1000_USA-road-d.NY.gr,value,43329.0


In [11]:
missing_val=ismissing.(r_dualisation)

mis_ind=findall(x->x==true,eachcol(missing_val))
println(r_dualisation[r_dualisation.name .=="1800_USA-road-d.COL.gr",:])

In [12]:
println(r_dualisation[r_dualisation.name .=="1800_USA-road-d.COL.gr",:])
select!(r_dualisation, Not(:nonuse))


1×3 DataFrame
 Row │ name                    nonuse    dual_val 
     │ String31                String7?  Float64? 
─────┼────────────────────────────────────────────
   1 │ 1800_USA-road-d.COL.gr  missing    missing 


Row,name,dual_val
,String31,Float64?
1,1000_USA-road-d.BAY.gr,48250.0
2,1000_USA-road-d.COL.gr,29672.8
3,1000_USA-road-d.NY.gr,43329.0
4,100_USA-road-d.BAY.gr,10857.1
5,100_USA-road-d.COL.gr,25320.2
6,100_USA-road-d.NY.gr,33931.0
7,1100_USA-road-d.BAY.gr,82422.6
8,1100_USA-road-d.COL.gr,42766.5
9,1100_USA-road-d.NY.gr,70089.2


In [13]:
size(r_stat,1)

123

In [14]:
st_dual=leftjoin(total, r_dualisation, on = :name)

Row,name,stat_value,time_stat,dual_val
,String31,Float64,Float64,Float64?
1,1000_USA-road-d.BAY.gr,40117.0,3.762,48250.0
2,1000_USA-road-d.COL.gr,24068.0,5.764,29672.8
3,1000_USA-road-d.NY.gr,36248.0,2.778,43329.0
4,100_USA-road-d.BAY.gr,8818.0,0.085,10857.1
5,100_USA-road-d.COL.gr,18930.0,0.0599999,25320.2
6,100_USA-road-d.NY.gr,15274.0,0.0470002,33931.0
7,1100_USA-road-d.BAY.gr,54157.0,5.131,82422.6
8,1100_USA-road-d.COL.gr,34075.0,9.959,42766.5
9,1100_USA-road-d.NY.gr,49351.0,6.309,70089.2


In [16]:
r_planCoupants1=DataFrame(CSV.File("result_planCoupant.csv",header=false),["name","pc_val","iter","time_pc"])
r_planCoupants1[1:2,:]

Row,name,pc_val,iter,time_pc
,String31,Float64,Int64,Float64
1,20_USA-road-d.BAY.gr,15332.6,45,7.495
2,20_USA-road-d.NY.gr,9454.47,15,1.049


In [17]:
st_dual_pc=leftjoin(st_dual,r_planCoupants1,on = :name)

Row,name,stat_value,time_stat,dual_val,pc_val,iter,time_pc
,String31,Float64,Float64,Float64?,Float64?,Int64?,Float64?
1,20_USA-road-d.BAY.gr,9365.0,0.013,15332.6,15332.6,45,7.495
2,20_USA-road-d.NY.gr,6848.0,0.00899982,9454.47,9454.47,15,1.049
3,40_USA-road-d.BAY.gr,7856.0,4.401,12664.3,12664.3,57,12.319
4,40_USA-road-d.COL.gr,10084.0,0.0190001,15059.0,15059.0,336,174.027
5,40_USA-road-d.NY.gr,12410.0,0.0250001,17330.1,17330.1,126,45.354
6,60_USA-road-d.BAY.gr,6965.0,0.053,10633.3,8571.81,120,73.277
7,60_USA-road-d.COL.gr,16904.0,0.023,23914.2,23914.2,75,21.971
8,60_USA-road-d.NY.gr,12244.0,0.0280001,31775.8,15809.6,222,180.397
9,1000_USA-road-d.BAY.gr,40117.0,3.762,48250.0,missing,missing,missing


In [46]:
r_planCoupants2=DataFrame(CSV.File("result_PC.csv",header=true))
r_planCoupants2[1:2,:]

Row,Name,Value,Nb_it,time
,String31,Float64,Int64,Float64
1,1000_USA-road-d.BAY.gr,45025.7,18,191.344
2,1000_USA-road-d.COL.gr,24119.2,14,192.392


In [51]:
select!(st_dual_pc, Not(:time_pc))

Row,name,stat_value,time_stat,dual_val,pc_val,iter,Value,Nb_it,time
,String31,Float64,Float64,Float64?,Float64?,Int64?,Float64?,Int64?,Float64?
1,20_USA-road-d.BAY.gr,9365.0,0.013,15332.6,15332.6,45,15332.6,45,1.639
2,20_USA-road-d.NY.gr,6848.0,0.00899982,9454.47,9454.47,15,9454.47,15,0.484
3,1000_USA-road-d.BAY.gr,40117.0,3.762,48250.0,missing,missing,45025.7,18,191.344
4,1000_USA-road-d.COL.gr,24068.0,5.764,29672.8,missing,missing,24119.2,14,192.392
5,1000_USA-road-d.NY.gr,36248.0,2.778,43329.0,missing,missing,37840.7,13,189.916
6,100_USA-road-d.BAY.gr,8818.0,0.085,10857.1,missing,missing,9884.89,83,194.116
7,100_USA-road-d.COL.gr,18930.0,0.0599999,25320.2,missing,missing,24397.5,108,180.222
8,100_USA-road-d.NY.gr,15274.0,0.0470002,33931.0,missing,missing,18951.8,100,180.127
9,1100_USA-road-d.BAY.gr,54157.0,5.131,82422.6,missing,missing,57818.0,5,189.854


In [60]:
st_dual_pc.nb_sommet=[parse(Int,split(x,"_")[1]) for x in st_dual_pc.name]


136-element Vector{Int64}:
   20
   20
 1000
 1000
 1000
  100
  100
  100
 1100
 1100
    ⋮
  850
  850
  850
  900
  900
  900
  950
  950
  950

In [61]:
st_dual_pc[1:5,:]

Row,name,stat_value,time_stat,dual_val,pc_val,iter,Value,Nb_it,time,nb_sommet
,String31,Float64,Float64,Float64?,Float64?,Int64?,Float64?,Int64?,Float64?,Int64
1,20_USA-road-d.BAY.gr,9365.0,0.013,15332.6,15332.6,45,15332.6,45,1.639,20
2,20_USA-road-d.NY.gr,6848.0,0.00899982,9454.47,9454.47,15,9454.47,15,0.484,20
3,1000_USA-road-d.BAY.gr,40117.0,3.762,48250.0,missing,missing,45025.7,18,191.344,1000
4,1000_USA-road-d.COL.gr,24068.0,5.764,29672.8,missing,missing,24119.2,14,192.392,1000
5,1000_USA-road-d.NY.gr,36248.0,2.778,43329.0,missing,missing,37840.7,13,189.916,1000


In [62]:
sort!(st_dual_pc, order(:nb_sommet))

Row,name,stat_value,time_stat,dual_val,pc_val,iter,Value,Nb_it,time,nb_sommet
,String31,Float64,Float64,Float64?,Float64?,Int64?,Float64?,Int64?,Float64?,Int64
1,20_USA-road-d.BAY.gr,9365.0,0.013,15332.6,15332.6,45,15332.6,45,1.639,20
2,20_USA-road-d.NY.gr,6848.0,0.00899982,9454.47,9454.47,15,9454.47,15,0.484,20
3,20_USA-road-d.COL.gr,5357.0,0.00999999,7076.52,missing,missing,7076.52,18,0.577,20
4,40_USA-road-d.BAY.gr,7856.0,4.401,12664.3,12664.3,57,missing,missing,missing,40
5,40_USA-road-d.COL.gr,10084.0,0.0190001,15059.0,15059.0,336,missing,missing,missing,40
6,40_USA-road-d.NY.gr,12410.0,0.0250001,17330.1,17330.1,126,missing,missing,missing,40
7,60_USA-road-d.BAY.gr,6965.0,0.053,10633.3,8571.81,120,missing,missing,missing,60
8,60_USA-road-d.COL.gr,16904.0,0.023,23914.2,23914.2,75,missing,missing,missing,60
9,60_USA-road-d.NY.gr,12244.0,0.0280001,31775.8,15809.6,222,missing,missing,missing,60


In [18]:
r_bc=DataFrame(CSV.File("result_BC.csv",header=false),["name","val_bc","status_bc","time"])

r_bc[1:3,:]

┌ Warning: thread = 1 warning: only found 1 / 4 columns around data row: 8. Filling remaining columns with `missing`
└ @ CSV C:\Users\Boss\.julia\packages\CSV\OnldF\src\file.jl:577
┌ Warning: thread = 1 warning: only found 1 / 4 columns around data row: 12. Filling remaining columns with `missing`
└ @ CSV C:\Users\Boss\.julia\packages\CSV\OnldF\src\file.jl:577
┌ Warning: thread = 1 warning: only found 1 / 4 columns around data row: 17. Filling remaining columns with `missing`
└ @ CSV C:\Users\Boss\.julia\packages\CSV\OnldF\src\file.jl:577
┌ Warning: thread = 1 warning: only found 1 / 4 columns around data row: 18. Filling remaining columns with `missing`
└ @ CSV C:\Users\Boss\.julia\packages\CSV\OnldF\src\file.jl:577
┌ Warning: thread = 1 warning: only found 1 / 4 columns around data row: 20. Filling remaining columns with `missing`
└ @ CSV C:\Users\Boss\.julia\packages\CSV\OnldF\src\file.jl:577
┌ Warning: thread = 1 warning: only found 1 / 4 columns around data row: 26. Filling remain

┌ Warning: thread = 1 warning: only found 1 / 4 columns around data row: 39. Filling remaining columns with `missing`
└ @ CSV C:\Users\Boss\.julia\packages\CSV\OnldF\src\file.jl:577


Row,name,val_bc,status_bc,time
,String31,Float64?,String15?,Float64?
1,20_USA-road-d.BAY.gr,9365.0,optimal,0.0370002
2,1000_USA-road-d.BAY.gr,40117.0,optimal,19.661
3,1000_USA-road-d.COL.gr,24068.0,optimal,17.625


In [19]:
stat_dual_bc=leftjoin(st_dual_pc,r_bc,on = :name)

Row,name,stat_value,time_stat,dual_val,pc_val,iter,time_pc,val_bc,status_bc,time
,String31,Float64,Float64,Float64?,Float64?,Int64?,Float64?,Float64?,String15?,Float64?
1,20_USA-road-d.BAY.gr,9365.0,0.013,15332.6,15332.6,45,7.495,9365.0,optimal,0.0370002
2,1000_USA-road-d.BAY.gr,40117.0,3.762,48250.0,missing,missing,missing,40117.0,optimal,19.661
3,1000_USA-road-d.COL.gr,24068.0,5.764,29672.8,missing,missing,missing,24068.0,optimal,17.625
4,1000_USA-road-d.NY.gr,36248.0,2.778,43329.0,missing,missing,missing,36248.0,optimal,16.181
5,100_USA-road-d.BAY.gr,8818.0,0.085,10857.1,missing,missing,missing,8818.0,optimal,0.134
6,100_USA-road-d.COL.gr,18930.0,0.0599999,25320.2,missing,missing,missing,18930.0,optimal,0.13
7,100_USA-road-d.NY.gr,15274.0,0.0470002,33931.0,missing,missing,missing,15274.0,optimal,0.245
8,1100_USA-road-d.COL.gr,34075.0,9.959,42766.5,missing,missing,missing,32664.0,optimal,49.66
9,1100_USA-road-d.NY.gr,49351.0,6.309,70089.2,missing,missing,missing,57273.0,feasible,107.976


In [23]:
gr=groupby(stat_dual_bc, :name)
for group in gr
    if size(group,1) > 1
        println(group)
    end
end


2×10 SubDataFrame
 Row │ name                    stat_value  time_stat  dual_val  pc_val    iter     time_pc   val_bc    status_bc  time     
     │ String31                Float64     Float64    Float64?  Float64?  Int64?   Float64?  Float64?  String15?  Float64? 
─────┼─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │ 1000_USA-road-d.BAY.gr     40117.0      3.762   48250.0   missing  missing   missing   40117.0  optimal      19.661
   2 │ 1000_USA-road-d.BAY.gr     40117.0      3.762   48250.0   missing  missing   missing   48250.0  feasible     95.908
2×10 SubDataFrame
 Row │ name                   stat_value  time_stat  dual_val  pc_val    iter     time_pc   val_bc    status_bc  time     
     │ String31               Float64     Float64    Float64?  Float64?  Int64?   Float64?  Float64?  String15?  Float64? 
─────┼──────────────────────────────────────────────────────────────────────────────────────────────

In [36]:
size(stat_dual_bc,1)

129

In [34]:
index_delete=findfirst((stat_dual_bc.name =="1100_USA-road-d.NY.gr") & (stat_dual_bc.time==107.976))


In [35]:
print(index_delete)

nothing

In [37]:
indexes_to_remove = filter(i -> length(unique(stat_dual_bc[stat_dual_bc.name .== stat_dual_bc[i, :name], :time])) > 1, 1:nrow(stat_dual_bc))

12-element Vector{Int64}:
  2
  4
  5
  6
  7
  9
 10
 11
 12
 13
 14
 15

In [38]:
stat_dual_bc[indexes_to_remove,:]

Row,name,stat_value,time_stat,dual_val,pc_val,iter,time_pc,val_bc,status_bc,time
,String31,Float64,Float64,Float64?,Float64?,Int64?,Float64?,Float64?,String15?,Float64?
1,1000_USA-road-d.BAY.gr,40117.0,3.762,48250.0,missing,missing,missing,40117.0,optimal,19.661
2,1000_USA-road-d.NY.gr,36248.0,2.778,43329.0,missing,missing,missing,36248.0,optimal,16.181
3,100_USA-road-d.BAY.gr,8818.0,0.085,10857.1,missing,missing,missing,8818.0,optimal,0.134
4,100_USA-road-d.COL.gr,18930.0,0.0599999,25320.2,missing,missing,missing,18930.0,optimal,0.13
5,100_USA-road-d.NY.gr,15274.0,0.0470002,33931.0,missing,missing,missing,15274.0,optimal,0.245
6,1100_USA-road-d.NY.gr,49351.0,6.309,70089.2,missing,missing,missing,57273.0,feasible,107.976
7,1000_USA-road-d.BAY.gr,40117.0,3.762,48250.0,missing,missing,missing,48250.0,feasible,95.908
8,1000_USA-road-d.NY.gr,36248.0,2.778,43329.0,missing,missing,missing,43329.0,feasible,91.117
9,100_USA-road-d.BAY.gr,8818.0,0.085,10857.1,missing,missing,missing,10857.1,feasible,80.168


In [41]:
df=stat_dual_bc[Not([2,9,5,6,7,4]),:]

Row,name,stat_value,time_stat,dual_val,pc_val,iter,time_pc,val_bc,status_bc,time
,String31,Float64,Float64,Float64?,Float64?,Int64?,Float64?,Float64?,String15?,Float64?
1,20_USA-road-d.BAY.gr,9365.0,0.013,15332.6,15332.6,45,7.495,9365.0,optimal,0.0370002
2,1000_USA-road-d.COL.gr,24068.0,5.764,29672.8,missing,missing,missing,24068.0,optimal,17.625
3,1100_USA-road-d.COL.gr,34075.0,9.959,42766.5,missing,missing,missing,32664.0,optimal,49.66
4,1000_USA-road-d.BAY.gr,40117.0,3.762,48250.0,missing,missing,missing,48250.0,feasible,95.908
5,1000_USA-road-d.NY.gr,36248.0,2.778,43329.0,missing,missing,missing,43329.0,feasible,91.117
6,100_USA-road-d.BAY.gr,8818.0,0.085,10857.1,missing,missing,missing,10857.1,feasible,80.168
7,100_USA-road-d.COL.gr,18930.0,0.0599999,25320.2,missing,missing,missing,25320.2,feasible,80.207
8,100_USA-road-d.NY.gr,15274.0,0.0470002,33931.0,missing,missing,missing,20939.9,feasible,80.13
9,1100_USA-road-d.NY.gr,49351.0,6.309,70089.2,missing,missing,missing,70089.2,feasible,96.49


In [43]:
r_pc=DataFrame(CSV.File("result_PC.csv",header=true))
r_pc[1:5,:]

Row,name,pc_val,iter,time_pc
,String31,Float64,Int64,Float64
1,1000_USA-road-d.BAY.gr,45025.7,18,191.344
2,1000_USA-road-d.COL.gr,24119.2,14,192.392
3,1000_USA-road-d.NY.gr,37840.7,13,189.916
4,100_USA-road-d.BAY.gr,9884.89,83,194.116
5,100_USA-road-d.COL.gr,24397.5,108,180.222


In [49]:
df_2=df

df_2.pc_val=

MethodError: MethodError: no method matching merge!(::DataFrame, ::DataFrame; on::Symbol)

Closest candidates are:
  merge!(!Matched::Union{Function, Type}, ::Any...) got unsupported keyword argument "on"
   @ Base abstractdict.jl:289
